In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
import pyspark.ml.feature
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline 

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.appName("Classifier").config('spark.executor.memory','8G').config("spark.driver.memory", "8G").getOrCreate()

21/11/04 19:10:54 WARN Utils: Your hostname, weiyuangdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.17.194.58 instead (on interface en0)
21/11/04 19:10:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/04 19:10:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv("politifact_with_users.csv",header=True, sep = ',',multiLine=True, escape = "\"",inferSchema =True)

In [4]:
df.show()

+------------------+----------------+--------------+-------------+----------+--------------------+--------+---------------+--------------------+--------------------+----+-------+---------------+-------------+------------+----------------+-----------+--------------+--------------------+---------------+
|          tweet_id|created_at_tweet|favorite_count|retweet_count|   user_id|            location|verified|followers_count|              source|                text|fake|news_id|created_at_user|friends_count|listed_count|favourites_count|geo_enabled|statuses_count|has_extended_profile|default_profile|
+------------------+----------------+--------------+-------------+----------+--------------------+--------+---------------+--------------------+--------------------+----+-------+---------------+-------------+------------+----------------+-----------+--------------+--------------------+---------------+
|         890268075|      1218988054|             0|            0|  15189613|              

In [5]:
#df = df.select('news_id')

In [6]:
df.dtypes

[('tweet_id', 'bigint'),
 ('created_at_tweet', 'int'),
 ('favorite_count', 'int'),
 ('retweet_count', 'int'),
 ('user_id', 'bigint'),
 ('location', 'string'),
 ('verified', 'boolean'),
 ('followers_count', 'double'),
 ('source', 'string'),
 ('text', 'string'),
 ('fake', 'double'),
 ('news_id', 'int'),
 ('created_at_user', 'double'),
 ('friends_count', 'double'),
 ('listed_count', 'double'),
 ('favourites_count', 'double'),
 ('geo_enabled', 'boolean'),
 ('statuses_count', 'double'),
 ('has_extended_profile', 'boolean'),
 ('default_profile', 'boolean')]

In [7]:
df = df.select('created_at_tweet','created_at_user','text','location','verified','source', 'followers_count','retweet_count','favorite_count','friends_count','listed_count','favourites_count','statuses_count','geo_enabled','has_extended_profile','default_profile','fake')
#df = df.select('created_at','text','location','verified','source', 'followers_count','retweet_count','favorite_count','fake')

In [8]:
#df = df.withColumn('created_at_tweet', to_timestamp(col('created_at_tweet'), 'yyyy-MM-dd HH:mm:ss'))
#df = df.withColumn('created_at_user', to_timestamp(col('created_at_user'), 'yyyy-MM-dd HH:mm:ss'))

In [9]:
#df = df.withColumn('created_at', df.created_at.cast("long"))

In [10]:
df.dtypes

[('created_at_tweet', 'int'),
 ('created_at_user', 'double'),
 ('text', 'string'),
 ('location', 'string'),
 ('verified', 'boolean'),
 ('source', 'string'),
 ('followers_count', 'double'),
 ('retweet_count', 'int'),
 ('favorite_count', 'int'),
 ('friends_count', 'double'),
 ('listed_count', 'double'),
 ('favourites_count', 'double'),
 ('statuses_count', 'double'),
 ('geo_enabled', 'boolean'),
 ('has_extended_profile', 'boolean'),
 ('default_profile', 'boolean'),
 ('fake', 'double')]

In [11]:
df = df.dropna(subset=('text'))
df = df.dropna(subset = ('geo_enabled','has_extended_profile','default_profile'))

In [12]:
df = df.fillna('null', subset = ('source', 'location'))

In [13]:
tokenizer_text = Tokenizer(inputCol='text',outputCol='text1')
stopwords_remover_text = StopWordsRemover(inputCol='text1',outputCol='text2')
vectorizer_text = CountVectorizer(inputCol='text2',outputCol='text3')
idf_text = IDF(inputCol='text3',outputCol='text_vc')

In [14]:
tokenizer_location = Tokenizer(inputCol='location',outputCol='location1')
stopwords_remover_location = StopWordsRemover(inputCol='location1',outputCol='location2')
vectorizer_location = CountVectorizer(inputCol='location2',outputCol='location3')
idf_location = IDF(inputCol='location3',outputCol='location_vc')

In [15]:
tokenizer_source = Tokenizer(inputCol='source',outputCol='source1')
stopwords_remover_source = StopWordsRemover(inputCol='source1',outputCol='source2')
vectorizer_source = CountVectorizer(inputCol='source2',outputCol='source3')
idf_source = IDF(inputCol='source3',outputCol='source_vc')

In [16]:
(trainDF,testDF) = df.randomSplit((0.7,0.3),seed=42)

In [17]:
input_cols = ['created_at_tweet','created_at_user','text_vc','location_vc','verified','source_vc', 'followers_count','retweet_count','favorite_count','friends_count','listed_count','favourites_count','statuses_count','geo_enabled','has_extended_profile','default_profile']

#input_cols = ['created_at','text_vc','location_vc','verified','source_vc', 'followers_count','retweet_count','favorite_count']

In [18]:
assembler = VectorAssembler(inputCols=input_cols,outputCol="features")

In [19]:
LR = LogisticRegression(featuresCol='features',labelCol='fake')

In [20]:
pipeline = Pipeline(stages=[tokenizer_text,stopwords_remover_text,vectorizer_text,idf_text,
                            tokenizer_source,stopwords_remover_source,vectorizer_source,idf_source,
                            tokenizer_location,stopwords_remover_location,vectorizer_location,idf_location,
                            assembler,LR])

In [21]:
lr_model = pipeline.fit(trainDF)

21/11/04 19:11:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/04 19:11:53 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:11 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/11/04 19:12:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/11/04 19:12:29 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:30 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:31 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:32 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:34 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/04 19:12:35 WARN DAGScheduler: Broadcasting large task binary with size 26.1 MiB
21/11/0

In [22]:
predictions = lr_model.transform(testDF)

In [23]:
predictions = predictions.select('fake','prediction')

In [24]:

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='fake',predictionCol='prediction',metricName='accuracy')
evaluator_f1 = MulticlassClassificationEvaluator(labelCol='fake',predictionCol='prediction',metricName='f1')
evaluator_precision = MulticlassClassificationEvaluator(labelCol='fake',predictionCol='prediction',metricName='weightedPrecision')
evaluator_recall = MulticlassClassificationEvaluator(labelCol='fake',predictionCol='prediction',metricName='weightedRecall')


In [25]:

print('Accuracy:', evaluator_accuracy.evaluate(predictions))
print('F1:',evaluator_f1.evaluate(predictions))
print('Precision:',evaluator_precision.evaluate(predictions))
print('Recall:',evaluator_recall.evaluate(predictions))


21/11/04 19:14:44 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


Accuracy: 0.960558151341832


21/11/04 19:14:54 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


F1: 0.9604506329496398


21/11/04 19:15:03 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


Precision: 0.9603931956678895


21/11/04 19:15:13 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


Recall: 0.960558151341832


In [26]:
TP = predictions[(predictions.prediction == 1) & (predictions.fake == 1)].count()
FN = predictions[(predictions.prediction == 0) & (predictions.fake == 1)].count()
TN = predictions[(predictions.prediction == 0) & (predictions.fake == 0)].count()
FP = predictions[(predictions.prediction == 1) & (predictions.fake == 0)].count()

21/11/04 19:15:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
21/11/04 19:15:24 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB
21/11/04 19:15:35 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB
21/11/04 19:15:45 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB
21/11/04 19:15:56 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


In [27]:
print('Accuracy:', (TP+TN)/(TP+FP+FN+TN))
print('Precision:', (TP)/(TP+FP))
print('Recall:', (TP)/(TP+FN))
print('F1:',(2*TP)/(2*TP+FP+FN))

Accuracy: 0.960558151341832
Precision: 0.933890508973935
Recall: 0.9180482903330673
F1: 0.9259016393442623
